<a href="https://colab.research.google.com/github/souma21122002/Internship2025/blob/main/Celeb_DF_Face%26Frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import zipfile
zip_ref=zipfile.ZipFile('/content/drive/MyDrive/Internship2025/Celeb_DF_Dataset/celeb_DF.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
!pip install mediapipe==0.10.20 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1

In [1]:
!pip install mediapipe==0.10.20
!pip install opencv-python


In [2]:

import cv2
import mediapipe as mp
import os
from google.colab.patches import cv2_imshow

# Define paths
input_folder = '/content/Celeb-real'
cropped_faces_folder = '/content/Celeb_real_Mediapipe/detected_faces_mediapipe/'
original_frames_folder = '/content/Celeb_real_Mediapipe/real_frames_mediapipe/'

# Create output directories if they don't exist
os.makedirs(cropped_faces_folder, exist_ok=True)
os.makedirs(original_frames_folder, exist_ok=True)

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
DETECTION_CONFIDENCE = 0.5

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=DETECTION_CONFIDENCE) as face_detector:
    for video_file in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_file)

        if video_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            print(f"Processing video: {video_file}")
            base_name = os.path.splitext(video_file)[0]

            # Create subfolders for each video
            video_cropped_folder = os.path.join(cropped_faces_folder, base_name)
            video_real_frame_folder = os.path.join(original_frames_folder, base_name)
            os.makedirs(video_cropped_folder, exist_ok=True)
            os.makedirs(video_real_frame_folder, exist_ok=True)

            cap = cv2.VideoCapture(video_path)
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1

                # Convert BGR to RGB for MediaPipe
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detector.process(rgb_frame)

                if results.detections:
                    # Save the full frame (original) if face is detected
                    real_frame_filename = f"{base_name}_frame_{frame_count:06d}.jpg"
                    real_frame_path = os.path.join(video_real_frame_folder, real_frame_filename)
                    cv2.imwrite(real_frame_path, frame)

                    # Save cropped faces
                    for i, detection in enumerate(results.detections):
                        bboxC = detection.location_data.relative_bounding_box
                        ih, iw, _ = frame.shape
                        x1 = int(bboxC.xmin * iw)
                        y1 = int(bboxC.ymin * ih)
                        w = int(bboxC.width * iw)
                        h = int(bboxC.height * ih)
                        x2 = x1 + w
                        y2 = y1 + h

                        # Ensure coordinates are within frame bounds
                        cropped_face = frame[max(0, y1):min(ih, y2), max(0, x1):min(iw, x2)]

                        if cropped_face.shape[0] > 0 and cropped_face.shape[1] > 0:
                            cropped_filename = f"{base_name}_frame_{frame_count:06d}_face_{i}.jpg"
                            cropped_path = os.path.join(video_cropped_folder, cropped_filename)
                            cv2.imwrite(cropped_path, cropped_face)

            cap.release()
            print(f"Finished processing: {video_file}")
            print(f"→ Real frames saved to: {video_real_frame_folder}")
            print(f"→ Cropped faces saved to: {video_cropped_folder}")

cv2.destroyAllWindows()

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.1, so it will not be used.
  warnings.warn(


Processing video: id44_0001.mp4
Finished processing: id44_0001.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/real_frames_mediapipe/id44_0001
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/detected_faces_mediapipe/id44_0001
Processing video: id33_0000.mp4
Finished processing: id33_0000.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/real_frames_mediapipe/id33_0000
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/detected_faces_mediapipe/id33_0000
Processing video: id51_0004.mp4
Finished processing: id51_0004.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/real_frames_mediapipe/id51_0004
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/detected_faces_mediapipe/id51_0004
Processing video: id30_0009.mp4
Finished processing: id30_0009.mp4
→ Real frames saved to: /content/Celeb_real_Mediapipe/real_frames_mediapipe/id30_0009
→ Cropped faces saved to: /content/Celeb_real_Mediapipe/detected_faces_mediapipe/id30_0009
Processing video: id27_0

In [5]:
import os
from collections import defaultdict
from pathlib import Path

# Base directory
base_dir = "/content/Celeb_real_Mediapipe"

# Paths to face and frame directories
faces_dir = os.path.join(base_dir, "detected_faces_mediapipe")
frames_dir = os.path.join(base_dir, "real_frames_mediapipe")

def extract_metadata(filename):
    """Extract video ID and frame number from filename"""
    parts = filename.split('_')
    video_id = '_'.join(parts[:2])  # e.g., 'id0_0000'
    frame_num = parts[3]  # e.g., '000001'
    return video_id, frame_num

# Build mapping of frames to faces
frame_to_faces = defaultdict(list)
missing_frames = set()
missing_faces = set()

# Iterate through all face files
for root, _, files in os.walk(faces_dir):
    for face_file in files:
        if face_file.endswith('.jpg'):
            try:
                video_id, frame_num = extract_metadata(face_file)
                frame_file = f"{video_id}_frame_{frame_num}.jpg"

                # Check if corresponding frame exists
                frame_path = os.path.join(frames_dir, video_id, frame_file)
                if not os.path.exists(frame_path):
                    missing_frames.add(frame_path)

                # Record the mapping
                frame_to_faces[(video_id, frame_num)].append(face_file)
            except Exception as e:
                print(f"Error processing {face_file}: {str(e)}")

# Check for frames that don't have any faces
for root, _, files in os.walk(frames_dir):
    for frame_file in files:
        if frame_file.endswith('.jpg'):
            try:
                video_id, frame_num = extract_metadata(frame_file)
                key = (video_id, frame_num)

                # If no faces exist for this frame
                if key not in frame_to_faces:
                    missing_faces.add(os.path.join(root, frame_file))
            except Exception as e:
                print(f"Error processing {frame_file}: {str(e)}")

# Print results
print("\n=== Face-Frame Mapping Report ===")
print(f"Total frames with faces: {len(frame_to_faces)}")
print(f"Total face images: {sum(len(faces) for faces in frame_to_faces.values())}")

print("\n=== Missing Frames (Faces without original frames) ===")
for missing in sorted(missing_frames)[:10]:  # Print first 10 for brevity
    print(missing)
print(f"... (Total {len(missing_frames)} missing frames)")

print("\n=== Missing Faces (Frames without any detected faces) ===")
for missing in sorted(missing_faces)[:10]:  # Print first 10 for brevity
    print(missing)
print(f"... (Total {len(missing_faces)} frames without faces)")

# Print per-video statistics
print("\n=== Per-Video Statistics ===")
video_stats = defaultdict(lambda: {'frames': 0, 'faces': 0})
for (video_id, _), faces in frame_to_faces.items():
    video_stats[video_id]['frames'] += 1
    video_stats[video_id]['faces'] += len(faces)

for video_id, stats in sorted(video_stats.items()):
    print(f"{video_id}: {stats['frames']} frames, {stats['faces']} faces")


=== Face-Frame Mapping Report ===
Total frames with faces: 222305
Total face images: 225545

=== Missing Frames (Faces without original frames) ===
... (Total 0 missing frames)

=== Missing Faces (Frames without any detected faces) ===
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000001.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000002.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000003.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000004.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000005.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000006.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000007.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_0000_frame_000008.jpg
/content/Celeb_real_Mediapipe/real_frames_mediapipe/id0_0000/id0_00

In [ ]:
from google.colab import files
import shutil
import os

dir_to_download = "/content/Celeb_real_Mediapipe"

output_zip = "Celeb_real_Mediapipe_Output.zip"
shutil.make_archive(output_zip[:-4], 'zip', dir_to_download)


files.download(output_zip)

In [4]:

import cv2
import mediapipe as mp
import os
from google.colab.patches import cv2_imshow

# Define paths
input_folder = '/content/Celeb-synthesis'
cropped_faces_folder = '/content/Celeb_systhesis_Mediapipe/detected_synthesis_faces_mediapipe/'
original_frames_folder = '/content/Celeb_systhesis_Mediapipe/systhesis_frames_mediapipe/'

# Create output directories if they don't exist
os.makedirs(cropped_faces_folder, exist_ok=True)
os.makedirs(original_frames_folder, exist_ok=True)

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
DETECTION_CONFIDENCE = 0.5

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=DETECTION_CONFIDENCE) as face_detector:
    for video_file in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_file)

        if video_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            print(f"Processing video: {video_file}")
            base_name = os.path.splitext(video_file)[0]

            # Create subfolders for each video
            video_cropped_folder = os.path.join(cropped_faces_folder, base_name)
            video_real_frame_folder = os.path.join(original_frames_folder, base_name)
            os.makedirs(video_cropped_folder, exist_ok=True)
            os.makedirs(video_real_frame_folder, exist_ok=True)

            cap = cv2.VideoCapture(video_path)
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1

                # Convert BGR to RGB for MediaPipe
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detector.process(rgb_frame)

                if results.detections:
                    # Save the full frame (original) if face is detected
                    real_frame_filename = f"{base_name}_frame_{frame_count:06d}.jpg"
                    real_frame_path = os.path.join(video_real_frame_folder, real_frame_filename)
                    cv2.imwrite(real_frame_path, frame)

                    # Save cropped faces
                    for i, detection in enumerate(results.detections):
                        bboxC = detection.location_data.relative_bounding_box
                        ih, iw, _ = frame.shape
                        x1 = int(bboxC.xmin * iw)
                        y1 = int(bboxC.ymin * ih)
                        w = int(bboxC.width * iw)
                        h = int(bboxC.height * ih)
                        x2 = x1 + w
                        y2 = y1 + h

                        # Ensure coordinates are within frame bounds
                        cropped_face = frame[max(0, y1):min(ih, y2), max(0, x1):min(iw, x2)]

                        if cropped_face.shape[0] > 0 and cropped_face.shape[1] > 0:
                            cropped_filename = f"{base_name}_frame_{frame_count:06d}_face_{i}.jpg"
                            cropped_path = os.path.join(video_cropped_folder, cropped_filename)
                            cv2.imwrite(cropped_path, cropped_face)

            cap.release()
            print(f"Finished processing: {video_file}")
            print(f"→ Real frames saved to: {video_real_frame_folder}")
            print(f"→ Cropped faces saved to: {video_cropped_folder}")

cv2.destroyAllWindows()

Processing video: id47_id43_0008.mp4
Finished processing: id47_id43_0008.mp4
→ Real frames saved to: /content/Celeb_systhesis_Mediapipe/systhesis_frames_mediapipe/id47_id43_0008
→ Cropped faces saved to: /content/Celeb_systhesis_Mediapipe/detected_synthesis_faces_mediapipe/id47_id43_0008
Processing video: id17_id6_0001.mp4
Finished processing: id17_id6_0001.mp4
→ Real frames saved to: /content/Celeb_systhesis_Mediapipe/systhesis_frames_mediapipe/id17_id6_0001
→ Cropped faces saved to: /content/Celeb_systhesis_Mediapipe/detected_synthesis_faces_mediapipe/id17_id6_0001
Processing video: id21_id17_0008.mp4
Finished processing: id21_id17_0008.mp4
→ Real frames saved to: /content/Celeb_systhesis_Mediapipe/systhesis_frames_mediapipe/id21_id17_0008
→ Cropped faces saved to: /content/Celeb_systhesis_Mediapipe/detected_synthesis_faces_mediapipe/id21_id17_0008
Processing video: id54_id49_0003.mp4
Finished processing: id54_id49_0003.mp4
→ Real frames saved to: /content/Celeb_systhesis_Mediapipe/s

KeyboardInterrupt: 